In [60]:
from keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.layers import Conv2D, LeakyReLU,MaxPooling2D,Flatten,Dense,Dropout
from keras.models import Model
import keras.utils as image
import numpy as np
import pandas as pd
import os
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split
import keras
import random

random.seed(0)
print(keras.__version__)

2.10.0


## Extract the features

### LeakyReLU 

In [66]:
# ## VGG16 model
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# # Replace ReLU with LeakyReLU
# x = base_model.input
# for layer in base_model.layers:
#     if isinstance(layer, Conv2D):
#         # Creates a new layer with the same configuration as the current layer, but without the activation function
#         new_layer = Conv2D(
#             filters=layer.filters,
#             kernel_size=layer.kernel_size,
#             strides=layer.strides,
#             padding=layer.padding,
#             activation=None,  
#             name=layer.name
#         )
        
#         new_layer.build(layer.input_shape)
#         new_layer.set_weights(layer.get_weights())
        
#         x = new_layer(x)
        
#         # add LeakyReLU
#         x = LeakyReLU(alpha=0.01)(x)
        
#     elif isinstance(layer, MaxPooling2D):
#         x = layer(x)


# # Add the full connection layer
# x = Flatten()(x)  
# x = Dense(4096)(x)
# x = LeakyReLU(alpha=0.01)(x)
# x = Dropout(0.5)(x)

# # feature vector 4096
# LeakyReLU_model = Model(inputs=base_model.input, outputs=x)



In [91]:
#Load images, preprocess, and extract features
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = model.predict(img_preprocessed)
    return features.flatten()
    #Extract the features and save the results to a CSV file

def process_images_and_save_features(source_dir, output_csv_file, model):
    data = {'filename': [], 'features': []}
    
    for subdir, _, files in os.walk(source_dir):
        category_name = os.path.basename(subdir)
        if files:
            print(f"Processing category: {category_name}")
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                img_path = os.path.join(subdir, file)
                features = extract_features(img_path, model)
                save_img_path = os.path.join('./images', subdir, file)
                data['filename'].append(save_img_path)
                data['features'].append(features)
    

    df = pd.DataFrame(data)
    df['features'] = df['features'].apply(lambda x: ','.join(x.astype(str)))  # Converts an array of features to a comma-separated string
    df.to_csv(output_csv_file, index=False)
    print("Features are successfully saved to CSV.")



base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3))
intermediate_layer_model = keras.Model(inputs=base_model.input,
                                       outputs=base_model.get_layer("fc1").output)

source_directory = './after-images'
output_csv_file = 'features_database.csv'
process_images_and_save_features(source_directory, output_csv_file, intermediate_layer_model)


In [15]:
df = pd.read_csv('LeakyReLU_features.csv')

print(df.head(10))

# Analyze statistical data for features, such as maximum, minimum, and average values
feature_stats = df['features'].apply(lambda x: pd.Series([float(i) for i in x.split(',')]))
print(feature_stats.describe())

                                            filename  \
0  ./after-images\001.Black_footed_Albatross\Blac...   
1  ./after-images\001.Black_footed_Albatross\Blac...   
2  ./after-images\001.Black_footed_Albatross\Blac...   
3  ./after-images\001.Black_footed_Albatross\Blac...   
4  ./after-images\001.Black_footed_Albatross\Blac...   
5  ./after-images\001.Black_footed_Albatross\Blac...   
6  ./after-images\001.Black_footed_Albatross\Blac...   
7  ./after-images\001.Black_footed_Albatross\Blac...   
8  ./after-images\001.Black_footed_Albatross\Blac...   
9  ./after-images\001.Black_footed_Albatross\Blac...   

                                            features  
0  -0.32282764,-0.33388615,-0.3026956,-0.16015178...  
1  -0.5151883,-0.40601066,-0.2200102,-0.104522504...  
2  -0.2909797,-0.3831334,-0.105868734,-0.16712645...  
3  -0.2644592,-0.3867885,-0.20360933,-0.03952134,...  
4  -0.5415482,-0.27620023,-0.35882914,-0.24081586...  
5  -0.23514183,-0.41969904,-0.05220923,-0.3252513... 

## Apply Approximate Nearest Neighbor Function

In [3]:
df = pd.read_csv('LeakyReLU_features.csv')

In [4]:
print(df['features'])

0        -0.14224967,-0.070818335,2.1476192,1.1205393,-...
1        -0.3246294,-0.3124694,27.85961,54.828472,9.991...
2        -0.18895955,-0.13565785,-0.039182447,8.250661,...
3        -0.17739522,-0.22490333,-0.16277212,3.9561818,...
4        -0.13278978,-0.3863413,4.439764,16.744944,-0.0...
                               ...                        
11783    0.48664144,26.14469,-0.08949319,-0.17218028,-0...
11784    -0.20380531,-0.3805666,-0.110192865,10.370926,...
11785    11.866622,13.224884,18.278954,11.173217,-0.141...
11786    -0.27159083,-0.20956694,-0.024098996,-0.023456...
11787    -0.68216264,-0.06175893,-0.20084643,16.13593,-...
Name: features, Length: 11788, dtype: object


In [90]:
import faiss
import numpy as np

features_df = df['features'].to_numpy()
features_all = []
for feat in features_df:
    feat = feat.split(',')
    feat = [float(f) for f in feat]
    features_all.append(feat)
features_all = np.array(features_all)
print(features_all[:5])


[[-0.14224967 -0.07081833  2.1476192  ... -0.2711366   2.5553672
  -0.04680335]
 [-0.3246294  -0.3124694  27.85961    ... -0.16064289 -0.29520825
  19.332577  ]
 [-0.18895955 -0.13565785 -0.03918245 ...  8.794187   10.094434
   1.8475256 ]
 [-0.17739522 -0.22490333 -0.16277212 ... -0.10267443 -0.13243131
  35.22717   ]
 [-0.13278978 -0.3863413   4.439764   ... -0.31834787 -0.12048921
  -0.16350059]]
11788


In [87]:
def extract_features(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)
    img_preprocessed = preprocess_input(img_array_expanded)
    features = model.predict(img_preprocessed)
    return features.flatten()

root = "./test_img/trial4"
files = os.listdir(root)
features_q = []
for f in files:
    path = os.path.join(root, f)
    if path.endswith(('.png', '.jpg')):
        feat = extract_features(path, intermediate_layer_model)
        features_q.append(feat)

features_q = np.array(features_q)


1/1 [==============================] - 0s 233ms/step


In [88]:
# query_vector = np.max(features_q, axis=0).reshape((1, 4096))
# print(query_vector.shape)
query_vector = features_q[2].reshape((1, 4096))
print(query_vector)

[[2.1145682  4.4280157  9.126083   ... 0.         0.         0.93021107]]


In [83]:
dimension = features_all.shape[1]
index = faiss.IndexFlatL2(dimension)  # Using L2 distance for similarity
index.add(features_all[:10])  # Add the dataset to the index

# Perform the search
k = 5  # Number of nearest neighbors to find
D, I = index.search(query_vector, k)  # D is the distance, I is the indices of the nearest neighbors

In [84]:
print(I)
print(D)
print(df.iat[I[0,0], 0])
print(df.iat[I[0,1], 0])
print(df.iat[I[0,2], 0])
print(df.iat[I[0,3], 0])
print(df.iat[I[0,4], 0])

[[8 6 5 0 4]]
[[270571.78 360135.4  459409.   465120.44 631355.1 ]]
./after-images\001.Black_footed_Albatross\Black_Footed_Albatross_0010_796097.jpg
./after-images\001.Black_footed_Albatross\Black_Footed_Albatross_0008_796083.jpg
./after-images\001.Black_footed_Albatross\Black_Footed_Albatross_0007_796138.jpg
./after-images\001.Black_footed_Albatross\Black_Footed_Albatross_0001_796111.jpg
./after-images\001.Black_footed_Albatross\Black_Footed_Albatross_0006_796065.jpg
